# Day 5 - Data pre-processing

In [ ]:
import yfinance as yf

import matplotlib.pyplot as plt
# Set default parameters for 'matplotlib' package
# More info on tweaking default parameters: https://matplotlib.org/stable/tutorials/introductory/customizing.html
plt.rcParams["figure.figsize"] = [10,8]  # Set default figure size

import pandas as pd
import datetime as dt
import numpy as np

In Python, you can suppress warnings using the `warnings` library. To ignore all warnings, you can use the `warnings.filterwarnings()` function with the argument `'ignore'`.

In [ ]:
import warnings
#warnings.filterwarnings('ignore')                          # Suppress all warnings
warnings.filterwarnings('ignore', module='yfinance')        # Suppress warnings from a specific library

In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', '^GSPC']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2010-01-01'
end_date = '2016-12-31'

# User yfinance to load the desired data. As simple as that.
panel_data = yf.download(tickers, start_date, end_date)

In [ ]:
panel_data.head(9)

In [ ]:
# Changing the format of the datetime index:
# NOTE: If you need to change the format of the datetime, you would have to convert the index to a string:
# check out different options here: https://www.programiz.com/python-programming/datetime/strftime 
panel_data_formatted=panel_data.copy() # just making a copy so I don't mess up the original df

panel_data_formatted.index=panel_data.index.strftime('%a, %d %b %Y')
panel_data_formatted

In [ ]:
print(type(panel_data.index))
print(type(panel_data_formatted.index))

In pandas, you **cannot** globally set the display format of datetime objects without converting them to strings, you can control the output format for specific output operations. 

You **can** change the format of the datetime but this involves actually converting the datetime object to string, which is not ideas if you rely on datetime functionality in your dataframe later on.  

In [ ]:
# Try selecting a subset of data on a specific date:
panel_data[panel_data.index=='2010-01-08']

# Try selecting a subset of data between specific dates:
panel_data[(panel_data.index>='2010-01-19') & (panel_data.index<='2010-01-29')];   # what does ';' at the end of the line do?

In [ ]:
ticker_list = {'INTC': 'Intel',
               'MSFT': 'Microsoft',
               'IBM': 'IBM',
               'BHP': 'BHP',
               'TM': 'Toyota',
               'AAPL': 'Apple',
               'AMZN': 'Amazon',
               'BA': 'Boeing',
               'QCOM': 'Qualcomm',
               'KO': 'Coca-Cola',
               'GOOG': 'Google',
               'SONY': 'Sony',
               'NFLX': 'Netflix'}

In [ ]:
def read_data(ticker_list,
          start=dt.datetime(2021, 1, 2),
          end=dt.datetime(2022, 2, 3)):
    """
    This function reads in closing price data from Yahoo
    for each tick in the ticker_list.
    """
    
    ticker = pd.DataFrame()

    for tick in ticker_list:
        prices = yf.download(tick, start, end)
        closing_prices = prices['Close']
        ticker[tick] = closing_prices

    return ticker

ticker = read_data(ticker_list)

In [ ]:
ticker.head(2) 
ticker.tail(2)

In [ ]:
# If instead of a dictionary, I provide a list of ticker symbols only, would my function `read_data()` still work?

ticker_list2 = ['INTC',
               'MSFT',
               'IBM',
               'BHP',
               'TM',
               'AAPL',
               'AMZN',
               'BA',
               'QCOM',
               'KO',
               'GOOG',
               'SONY',
               'NFLX']
ticker2 = read_data(ticker_list)
ticker2;

> <font color=DeepPink>Exercise 1</font>: 
> Using `head()` and `tail()` commands in reference to `Pandas.DataFrame` allows to view top and bottom rows. Declaring dataframe's name, e.g., `ticker`, displays the top 5 and the bottom 5 rows.<br>
> (a) Customise it to display a specific number of rows at the top and bottom simultaneously (that is as a single output). <br>
> (b) Customise `read_data()` function so that the dates are displayed as **yyyy-mm-dd** by defualt or the time format can be specified by the user.

## Access specific elements

In [ ]:
ticker.index[2:5]

In [ ]:
ticker.index[range(2,5)]

In [ ]:
ticker.loc[ticker.index[2:5], ['BHP', 'KO']]  

In [ ]:
# Access only those elements that fit a condition

ticker.loc[ticker.BHP<53.2, ['BHP', 'KO']]#.head(10)

To get a [five-point summary](https://en.wikipedia.org/wiki/Five-number_summary) statistics in a form of a table, use `.describe()` functionality (see [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) for customizing its call). Later, we will see a graphical way of representing these statistics via [Box plot](https://en.wikipedia.org/wiki/Box_plot).

`DataFrame.describe(percentiles=None, include=None, exclude=None, datetime_is_numeric=False)`

- `percentiles` : list-like of numbers (optional). The percentiles to include in the output. All should fall between 0 and 1. The default is [.25, .5, .75], which returns the 25th, 50th, and 75th percentiles.


In [ ]:
ticker.describe()

In [ ]:
ticker.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95], include=[np.number])

> <font color=DeepPink>Exercise 2</font>:<br>
> (a) How to filter out which columns to exclude from statistical summary?

In [ ]:
ticker['MSFT'].head()

In [ ]:
ticker.plot(title=ticker_list,subplots=True,figsize=(12,25))
#plt.show()

In [ ]:
#ticker.plot(title=ticker_list,subplots=False,figsize=(12,8)) # This line of code doesn't work very well. Why?
ticker.plot(title='Prices',subplots=False,figsize=(12,8)) 

# Note the change in figure size above

What is wrong with the picture above? How useful do you think it is for comparing stock performance?

Let's try plotting returns instead to see if we can get more insight...

In [ ]:
ret = ticker.pct_change()
ret.plot(title='Returns',figsize=(12,8))

In [ ]:
((1 + ret).cumprod() - 1).plot(title='Cumulative Returns',figsize=(12,8))

In [ ]:
type(ticker)

In [ ]:
isinstance(ticker,pd.DataFrame)

In [ ]:
def pdPlotReturns(x):
    if isinstance(x,pd.DataFrame):
        ret = ticker.pct_change()
        ((1 + ret).cumprod() - 1).plot(title='Cumulative Returns',figsize=(12,8))
    else:
        print(f'The variable provided by the user is of type {type(x)} but it should be Pandas DataFrame.')   


In [ ]:
pdPlotReturns(ticker)

The time period is not sufficient to compare long-run performance among the stocks we selected. Let's try to extend our time window:

In [ ]:
# Recall our previously defined function:
ticker = read_data(ticker_list)
ticker

In [ ]:
# Let's get more data
ticker = read_data(ticker_list, start=dt.datetime(2016, 1, 2), end=dt.datetime(2023, 2, 3))
ticker

In [ ]:
pdPlotReturns(ticker)

> ### <font color=red>Homework</font>:
> * Modify `pdPlotReturns()` function above to allow for cumulative returns to be returned as an output in addition to displaying a plot. 

## Dealing with missing data

### Load example data
Note the possible options:
1. Loading data from file outside of current directory.
2. Loading data directly from the web via link to file.

In [ ]:
#data = pd.read_csv("../../Data/nba.csv") # note relative path specification
data = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
data.head(20)

In [ ]:
# removing null values to avoid errors  
data.dropna(inplace = False)  # What does "inplace=False" mean?
data.head(20)

In [ ]:
# removing null values to avoid errors  
data.dropna(inplace = True)  # What does "inplace=True" mean?
  
# percentile list 
perc =[.20, .40, .60, .80] 
  
# list of dtypes to include 
include =['object', 'float', 'int'] 
  
# calling describe method 
desc = data.describe(percentiles = perc, include = include) 
  
# display 
desc 

As shown in the output, statistical description of dataframe was returned with the respective passed percentiles. For the columns with strings (permitted to be included via `Object` data type), `NaN` was returned for numeric operations.

In [ ]:
data.describe(percentiles = perc, include = ['float', 'int'] )

In [ ]:
data["Name"].describe() 

### Explore your data
In the example above, we were very quick to drop missing data. Lets explore how much data did we lose.

In [ ]:
# reload data
data = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")

In [ ]:
# Number of missing entries by variable name
data.isna().sum()

Now count the percentage of missing values for each column, simply by dividing the previous result by the length of the dataset (`len(df)`) and multiplying per 100.

In [ ]:
data.isna().sum()/len(data)*100

> ### <font color=red>Homework</font>: Format the output of the above line so that it
> - displays only 2 decimal points
> - (more challenging) also add `%` symbol at the end of each number to denote percentage.

When dealing with missing values, different alternatives can be applied:
- check the source, for example by contacting the data source to correct the missing values
    - a problem with missing values could also occur if you are loading data from a text file in different text encoding standard or language. Letters with accent grave, apostrophes, carets, tildes may present some issues
- drop missing values
- replace the missing value with a value
- leave the missing value as it is

**Droping missing values** can be one of the following alternatives:

- remove rows having missing values
- remove the whole column containing missing values 

We can use the `dropna()` by specifying the axis to be considered. If we set `axis = 0` (by apply the function `data.dropna(axis=0)`) we drop the entire row, if we set `axis = 1` we drop the whole column (apply the function `data.dropna(axis=1)`. 

**<font color=blue>Important:</font>** However, removed values are not applied to the original dataframe, but only to the result. We can use the argument `inplace=True` in order to store changes in the original dataframe `data`. 

In [ ]:
data

In [ ]:
data.dropna(axis=0)

In [ ]:
data.dropna(axis=1)

In [ ]:
data

In [ ]:
data.dropna(axis=0,inplace=True)

In [ ]:
data

> ## <font color=red>Homework (just for fun if you have time this weekend):</font> 
The [link](https://towardsdatascience.com/how-to-extract-text-from-pdf-245482a96de7) contains reviews on some Python libraries that claim to read text directly from PDF files. Pick one or try a few, see how it perfroms. I have not yet tested this functionality in Python, but have been using it through other programming languages. 
`pdfPlumber` looks promissing and Kenny said "`PyMuPDF` works for me. But, its syntax doesn't feel pythonic to me. `-m pip install PyMuPDF` then `import fitz` to start."